In [3]:
import pandas as pd
import os 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler

import torch
from torch import  nn
from torchvision import transforms ,datasets
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.optim.lr_scheduler as lr_scheduler

device = "cuda" if torch.cuda.is_available() else "cpu"
device

import cv2

In [4]:
images=np.load("/home/yash/Downloads/Dataset PAAC/Galaxy10 datasets/DECals/images-001.npy")
labels=np.load("/home/yash/Downloads/Dataset PAAC/Galaxy10 datasets/DECals/labels.npy")

In [5]:
images.shape

(17736, 256, 256, 3)

In [6]:
split_iamges=np.array_split(images,739)

In [7]:
np.shape(split_iamges)

(739, 24, 256, 256, 3)

In [8]:
transform=transforms.ToTensor()

In [5]:
transform=transforms.ToTensor()
images_ten=[]
for image in images:
    images_ten.append(transform(image))

In [ ]:
i=0
for image in images:
    ScaledImages[i]=(((image-np.min(image))/(np.max(image)-np.min(image))))
    i+=1

In [3]:
class Autoencoders(nn.Module):
    def __init__(self):
        super().__init__()
        self.Encoders=nn.Sequential(
            nn.Conv2d(3,16,3),
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.Conv2d(32,64,8),
            nn.ReLU(),
            nn.Conv2d(64,32,4),
            nn.ReLU(),
            nn.Conv2d(32,16,5),
            nn.ReLU(),
            nn.Conv2d(16,4,6)
        )

        self.Decoders=nn.Sequential(
            nn.ConvTranspose2d(4,16,6),
            nn.ReLU(),
            nn.ConvTranspose2d(16,32,5),
            nn.ReLU(),
            nn.ConvTranspose2d(32,64,4),
            nn.ReLU(),
            nn.ConvTranspose2d(64,32,8),
            nn.ReLU(),
            nn.ConvTranspose2d(32,16,5),
            nn.ReLU(),
            nn.ConvTranspose2d(16,3,3),
            nn.Sigmoid()
        )

    def forward(self,Dataset):
        Encoded=self.Encoders(Dataset)
        Decoded=self.Decoders(Encoded)
        return(Decoded)

In [4]:
model=Autoencoders().to(device)
LossFunc=nn.MSELoss()
Optm=Adam(params=model.parameters(),lr=1e-3)

In [9]:
epochs=20
outputs=[]

for epoch in range(epochs):
    for listarray in split_iamges:
        listarray=np.array(listarray)
        listarray=transform(listarray)
        listarray=listarray.to(device)

        genimg=model(listarray)
        loss=LossFunc(genimg,listarray)

        Optm.zero_grad()
        loss.backward()
        Optm.step()

        print(f"Epochs:{epoch+1},Loss:{loss.item():4f}")
        outputs.append((epoch,listarray,genimg))
        
    

ValueError: pic should be 2/3 dimensional. Got 4 dimensions.